In [6]:
from tensorflow.python.client import device_lib
from keras.models import Sequential
from keras.layers import Dropout, Dense
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from scipy import stats
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split, KFold
from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import StandardScaler
stop = set(stopwords.words('english'))
import os
import xgboost as xgb
import lightgbm as lgb
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib
import json
import ast
import eli5
from functools import reduce
import warnings
from sklearn.metrics import mean_squared_error
from scipy import stats
from math import sqrt
#from lightgbm import plot_tree
from hyperopt import hp, tpe
from hyperopt.fmin import fmin
from hyperopt import Trials
from hyperopt import fmin
from hyperopt import STATUS_OK
from hyperopt.pyll.stochastic import sample
import time
import gc
warnings.filterwarnings('ignore')
#import shap
print(device_lib.list_local_devices())

c:\users\chadchang\appdata\local\programs\python\python37\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16537209064671778951
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9117697966
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18344537225999046372
physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [17]:
from keras import backend
def rmse(y_true, y_pred):
    return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

train = pd.read_csv('FE_train.csv')
test  = pd.read_csv('FE_test.csv')
data  = pd.concat([train, test], axis=0)


#drop outlier
train.drop(train[(train['land_area'] > 1500) | (train['building_area'] >1000)].index, inplace= True)


train['total_price_log'] = np.log1p(train['total_price'])
features = [i for i in train.columns if i not in ['building_id', 'total_price','total_price_log', 'city', 'location_2']] 
X = train[features].values
Y = train['total_price_log'].values

model = Sequential()
model.add(Dense(128, kernel_initializer='normal',input_dim = train[features].shape[1], activation='relu'))

# The Hidden Layers :
model.add(Dense(128, kernel_initializer='normal',activation='relu'))
model.add(Dense(128, kernel_initializer='normal',activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, kernel_initializer='normal',activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, kernel_initializer='normal',activation='relu'))


# The Output Layer :
model.add(Dense(1, kernel_initializer='normal'))

model.compile(loss=rmse, optimizer='adam', metrics=[rmse])
model.fit(X, Y, epochs = 500, batch_size = 5000, validation_split = 0.2, verbose=2)


In [43]:
#train = pd.read_csv('dataset-0510/train.csv')
#test  = pd.read_csv('dataset-0510/test.csv')

train = pd.read_csv('FE_train.csv')
test  = pd.read_csv('FE_test.csv')
data  = pd.concat([train, test], axis=0)


#drop outlier
train.drop(train[(train['land_area'] > 1500) | (train['building_area'] >1000)].index, inplace= True)
'''
target_df = train.groupby(['city', 'town', 'building_type']).agg({'building_area' : ['mean', 'median'], 'land_area' : ['mean', 'median'], 'total_price' : ['mean', 'median']}).reset_index()
target_df.columns = [i[0] + '_' + i[1]  if i[1] != '' else i[0] for i in target_df.columns.tolist()]

target_df['total_price_median'] = np.log1p(target_df['total_price_median'])
target_df['price_land_rate_median'] = target_df['total_price_median'] / target_df['land_area_median']
target_df['price_building_rate_median'] = target_df['total_price_median'] / target_df['building_area_median']


combine_cols = ['city', 'town', 'building_type', 'price_land_rate_median', 'price_building_rate_median']
train = pd.merge(train, target_df[combine_cols], on =['city', 'town', 'building_type'], how='left')
test = pd.merge(test, target_df[combine_cols], on =['city', 'town', 'building_type'], how='left')


target_df = train.groupby('building_type')['total_price'].median().reset_index()
target_df['building_total_price'] = np.log1p(target_df['total_price'])
target_df.drop('total_price', axis =1, inplace = True)
train = pd.merge(train, target_df, on =['building_type'], how='left')
test = pd.merge(test, target_df, on =['building_type'], how='left')

train.loc[train['building_area'] == 4, 'parking_area'] = train.loc[train['building_area'] == 4, 'building_area'] / train.loc[train['building_area'] == 4, 'total_floor']
test.loc[train['building_area'] == 4, 'parking_area'] = test.loc[test['building_area'] == 4, 'building_area'] / test.loc[test['building_area'] == 4, 'total_floor']
'''
#drop_cols = [i for i in train.columns if np.sum(train[i]) == 60000 and 'index' in i]
train.drop([ 'inter_btw_building_type_building_use',
 'inter_btw_building_type_building_material',
 'inter_btw_building_type_parking_way',
 'inter_btw_building_use_building_material',
 'inter_btw_building_use_parking_way',
 'inter_btw_building_material_parking_way', 'building_type', 'parking_way','building_material', 'building_use'], axis = 1, inplace = True)
test.drop([ 'inter_btw_building_type_building_use',
 'inter_btw_building_type_building_material',
 'inter_btw_building_type_parking_way',
 'inter_btw_building_use_building_material',
 'inter_btw_building_use_parking_way',
 'inter_btw_building_material_parking_way', 'building_type', 'parking_way','building_material', 'building_use'], axis = 1, inplace = True)
#train.drop(drop_cols, axis = 1, inplace = True)
#test.drop(drop_cols, axis = 1, inplace = True)
gc.collect()

196

In [50]:
train['total_price_log'] = np.log1p(train['total_price'])
features = [i for i in train.columns if i not in ['building_id', 'total_price','total_price_log', 'city', 'location_2']] 
X = train[features].values
Y = train['total_price_log'].values

model = Sequential()
model.add(Dense(128, kernel_initializer='uniform',input_dim = train[features].shape[1], activation='sigmoid'))

# The Hidden Layers :
model.add(Dense(128, kernel_initializer='uniform',activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(500, kernel_initializer='uniform',activation='sigmoid'))

# The Output Layer :
model.add(Dense(1, kernel_initializer='normal'))

model.compile(loss=rmse, optimizer='adam', metrics=[rmse])
model.fit(X, Y, epochs = 500, batch_size = 5000, validation_split = 0.2, verbose=2)


Train on 47991 samples, validate on 11998 samples
Epoch 1/500
 - 3s - loss: 12.3307 - rmse: 12.3307 - val_loss: 8.3466 - val_rmse: 8.3466
Epoch 2/500
 - 0s - loss: 5.3546 - rmse: 5.3546 - val_loss: 1.6939 - val_rmse: 1.6939
Epoch 3/500
 - 0s - loss: 1.2709 - rmse: 1.2709 - val_loss: 1.6493 - val_rmse: 1.6493
Epoch 4/500
 - 0s - loss: 1.3879 - rmse: 1.3879 - val_loss: 0.9938 - val_rmse: 0.9938
Epoch 5/500
 - 0s - loss: 0.9720 - rmse: 0.9720 - val_loss: 0.9972 - val_rmse: 0.9972
Epoch 6/500
 - 0s - loss: 0.9731 - rmse: 0.9731 - val_loss: 0.9270 - val_rmse: 0.9270
Epoch 7/500
 - 0s - loss: 0.9389 - rmse: 0.9389 - val_loss: 0.9415 - val_rmse: 0.9415
Epoch 8/500
 - 0s - loss: 0.9260 - rmse: 0.9260 - val_loss: 0.9193 - val_rmse: 0.9193
Epoch 9/500
 - 0s - loss: 0.9265 - rmse: 0.9265 - val_loss: 0.9220 - val_rmse: 0.9220
Epoch 10/500
 - 0s - loss: 0.9220 - rmse: 0.9220 - val_loss: 0.9230 - val_rmse: 0.9230
Epoch 11/500
 - 0s - loss: 0.9212 - rmse: 0.9212 - val_loss: 0.9202 - val_rmse: 0.9202


KeyboardInterrupt: 

In [28]:
temp = model.predict(X)

In [31]:
train['total_price_log']

0        13.381036
1        15.015913
2        16.074236
3        16.469809
4        13.544637
5        12.890313
6        15.956787
7        14.333368
8        13.802448
9        16.284915
10       15.849286
11       16.235043
12       14.987657
13       15.385595
14       15.829483
15       16.168449
16       14.861726
17       15.236485
18       15.690540
19       15.616006
20       16.425662
21       15.496905
22       15.471942
23       15.331362
24       14.836590
25       15.537644
26       17.785623
27       13.905023
28       14.891543
29       15.616006
           ...    
59959    13.543883
59960    13.940832
59961    15.429368
59962    15.616009
59963    16.380187
59964    15.114354
59965    16.790066
59966    15.719361
59967    13.728808
59968    15.829555
59969    15.480295
59970    16.342804
59971    14.704439
59972    15.631207
59973    13.306724
59974    16.380188
59975    13.836441
59976    17.071690
59977    14.825907
59978    17.442378
59979    16.936171
59980    15.